In [1]:
import fitz
import re

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    doc = fitz.open(pdf_path)
    full_text = ""
    
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text = page.get_text()
        full_text += text + "\n"
    
    doc.close()
    
    # Clean the text
    full_text = re.sub(r'\s+', ' ', full_text)
    full_text = re.sub(r'\n+', '\n', full_text)
    
    return full_text.strip()

In [2]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning, module='colbert')


In [3]:
def chunk_text(text, chunk_size=512, overlap=128):
    """
    Split text into overlapping chunks
    chunk_size: approximate number of words per chunk
    overlap: number of overlapping words between chunks
    """
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        if chunk:
            chunks.append(chunk)
    
    return chunks

In [4]:
from colbert import Indexer
from colbert.infra import Run, RunConfig
from colbert.data import Queries, Collection

def initialize_colbert(index_name="pdf_index", experiment_name="pdf_experiment"):
    """Initialize ColBERT with configuration"""
    
    # Configure ColBERT run
    with Run().context(RunConfig(nranks=1, experiment=experiment_name)):
        config = {
            'doc_maxlen': 512,  # Max document length in tokens
            'query_maxlen': 128,  # Max query length in tokens
            'dim': 256,  # Embedding dimension
            'similarity': 'cosine',
            'checkpoint': 'colbert-ir/colbertv2.0'  # Use ColBERT v2.0 checkpoint
        }
        
    return config, index_name

/home/alisher/projects/america_ahhahh/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def prepare_collection(pdf_path):
    """Prepare document collection from PDF"""
    
    # Extract text
    text = extract_text_from_pdf(pdf_path)
    
    # Chunk the text
    chunks = chunk_text(text, chunk_size=512, overlap=128)
    
    # Create a collection with metadata
    collection = []
    metadata = []
    
    for idx, chunk in enumerate(chunks):
        collection.append(chunk)
        metadata.append({
            'chunk_id': idx,
            'source': pdf_path,
            'text': chunk[:100] + '...'  # Store preview
        })
    
    return collection, metadata

In [6]:
from colbert import Indexer
from colbert.infra import Run, RunConfig, ColBERTConfig
import torch

def index_documents(collection, index_name="pdf_index"):
    """Index the document collection using ColBERT"""
    
    # Check if CUDA is available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    if device == "cuda":
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    with Run().context(RunConfig(nranks=1, gpus=1)):
        config = ColBERTConfig(
            doc_maxlen=512,
            nbits=2,  # Compression bits
            kmeans_niters=4,
            checkpoint='colbert-ir/colbertv2.0'
        )
        
        indexer = Indexer(checkpoint="colbert-ir/colbertv2.0", config=config)
        
        # Create index
        indexer.index(
            name=index_name,
            collection=collection,
            overwrite=True
        )
        
        indexer.get_index()
    
    return indexer

In [7]:
from colbert import Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
import torch

def search_pdf(query, index_name="pdf_index", k=5):
    """Search the indexed PDF content"""
    
    # Check if CUDA is available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device for search: {device}")
    
    with Run().context(RunConfig(nranks=1, gpus=1)):
        config = ColBERTConfig(
            checkpoint='colbert-ir/colbertv2.0'
        )
        
        searcher = Searcher(index=index_name, config=config)
        
        # Search - returns a Ranking object with passage_ids and scores
        results = searcher.search(query, k=k)
        
        # Format results
        search_results = []
        
        # ColBERT returns a list where each item is (passage_id, rank, score)
        # or just the ranking indices
        for rank, (passage_id, score) in enumerate(zip(results[0], results[1])):
            search_results.append({
                'passage_id': passage_id,
                'rank': rank + 1,
                'score': score
            })
        
    return search_results

In [9]:
def embed_pdf_with_colbert(pdf_path, index_name="pdf_index"):
    """Complete pipeline to embed PDF with ColBERT v2.0"""
    
    print("Step 1: Extracting text from PDF...")
    text = extract_text_from_pdf(pdf_path)
    
    print("Step 2: Chunking document...")
    chunks = chunk_text(text, chunk_size=256, overlap=32)
    print(f"Created {len(chunks)} chunks")
    
    print("Step 3: Initializing ColBERT...")
    config, index_name = initialize_colbert(index_name=index_name)
    
    print("Step 4: Preparing collection...")
    collection, metadata = prepare_collection(pdf_path)
    
    print("Step 5: Indexing documents...")
    indexer = index_documents(collection, index_name)
    
    print("Step 6: Index created successfully!")
    
    # Save metadata for later reference
    import json
    with open(f"{index_name}_metadata.json", "w") as f:
        json.dump(metadata, f)
    
    return index_name, metadata

In [ ]:
if __name__ == "__main__":
    pdf_file = "asda.pdf"
    index_name, metadata = embed_pdf_with_colbert(pdf_file)

In [11]:
query = "Efficiency of AI"
results = search_pdf(query, index_name=index_name, k=5)

# Display results with metadata
for result in results:
    passage_id = result['passage_id']
    print(f"Rank {result['rank']}: Score {result['score']:.4f}")
    print(f"Content: {metadata[passage_id]['text']}")
    print("-" * 50)

Using device for search: cuda
[Nov 02, 17:28:49] #> Loading codec...
[Nov 02, 17:28:49] #> Loading IVF...
[Nov 02, 17:28:49] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3912.60it/s]

[Nov 02, 17:28:49] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 1073.54it/s]


#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: Efficiency of AI, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 8122, 1997, 9932,  102,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

Rank 1: Score 1.0000
Content: and in some settings, language model agents even outperformed humans in programming tasks with limit...
--------------------------------------------------
Rank 2: Score 2.0000
Content: What a year 2024 has been for AI. The recognition of AI’s role in advancing humanity’s knowledge is ...
--------------------------------------------------
Rank 3: Score 3.0000
Content: (cont’d) 6. Global AI optimism is rising—but dee

In [17]:
metadata[1]

{'chunk_id': 1,
 'source': 'asda.pdf',
 'text': 'What a year 2024 has been for AI. The recognition of AI’s role in advancing humanity’s knowledge is ...'}